In [1]:
import nltk
nltk.download('nps_chat')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa
from sklearn.metrics import accuracy_score
import torch
import pandas as pd
from datetime import datetime
import torchvision
from torch.utils.tensorboard import SummaryWriter
from transformers import RobertaTokenizerFast,RobertaTokenizer, RobertaForSequenceClassification,RobertaForMaskedLM, TextClassificationPipeline,TFRobertaModel
import numpy as np


[nltk_data] Downloading package nps_chat to
[nltk_data]     C:\Users\Emiel\AppData\Roaming\nltk_data...
[nltk_data]   Package nps_chat is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Emiel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Emiel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Emiel\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
posts = nltk.corpus.nps_chat.xml_posts()
X = [post.text for post in posts]
y  = ['question' if (post.get('class') == 'whQuestion' or post.get('class') == 'ynQuestion') else "other" for post in posts]
translate_dict = {'question':1, 'other':0}

y = [translate_dict[entry] for entry in y]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify = y)

In [3]:

def dialogue_act_features(post):
    features = {}
    for word in nltk.word_tokenize(post):
        features['contains({})'.format(word.lower())] = True
    return features


featuresets = [(dialogue_act_features(post.text), target) for post, target in list(zip(posts,y))]
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.8731060606060606


In [4]:
set([post.get('class') for post in posts])

{'Accept',
 'Bye',
 'Clarify',
 'Continuer',
 'Emotion',
 'Emphasis',
 'Greet',
 'Other',
 'Reject',
 'Statement',
 'System',
 'nAnswer',
 'whQuestion',
 'yAnswer',
 'ynQuestion'}

In [5]:
classifier.show_most_informative_features(50)

Most Informative Features
             contains(?) = True                1 : 0      =     59.5 : 1.0
           contains(wtf) = True                1 : 0      =     56.8 : 1.0
         contains(whats) = True                1 : 0      =     54.6 : 1.0
       contains(anybody) = True                1 : 0      =     44.8 : 1.0
           contains(asl) = True                1 : 0      =     44.8 : 1.0
            contains(tx) = True                1 : 0      =     32.9 : 1.0
         contains(which) = True                1 : 0      =     32.9 : 1.0
           contains(who) = True                1 : 0      =     31.1 : 1.0
          contains(part) = True                0 : 1      =     30.0 : 1.0
          contains(doin) = True                1 : 0      =     26.9 : 1.0
        contains(purple) = True                1 : 0      =     26.9 : 1.0
           contains(how) = True                1 : 0      =     25.0 : 1.0
        contains(anyone) = True                1 : 0      =     23.4 : 1.0

In [6]:
line = "Do you enjoy high performance programming?"
print(classifier.classify(dialogue_act_features(line)))


1


In [7]:
line = "What do you think about high performance programming?"
print(classifier.classify(dialogue_act_features(line)))

1


In [8]:
line = "i am testing this"
print(classifier.classify(dialogue_act_features(line)))

0
